# Import

In [1]:
from datetime import datetime 
import pandas as pd
import pickle
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon
from geopy import distance
from random import randint
import requests

# Data 

## Platform Type 

In [2]:
platform_type = randint(0,4)
platform_type 

0

## Time

In [3]:
time_of_order = datetime.now(tz=None)

In [4]:
day = 15
month = 6
if len(str(month)) == 1 : 
    month = f'0{month}'
    
year = time_of_order.year
pick_up_hour = 14
pick_up_min = 30
date = f"{day}/{month}/{year}"

Personnal_or_Business = "Personnal"
if Personnal_or_Business == "Personnal" : 
    Personnal_or_Business = 1
else : 
    Personnal_or_Business = 0



In [5]:
date = f"{day}/{month}/{year}"
date_time_obj = datetime.strptime(date, '%d/%m/%Y')

In [70]:
day_of_week = date_time_obj.weekday()


In [7]:
def is_week_end(day_of_week):
    if day_of_week > 5 : 
        week_end = 1
    else:  
        week_end = 0
    
    return week_end

In [8]:
week_end = is_week_end(day_of_week)
week_end

0

In [9]:
if (pick_up_hour >6 and pick_up_hour <9) or (pick_up_hour >16 and pick_up_hour <19) : 
    rush_hour = 1
else : 
    rush_hour = 0
    

## Location 

In [10]:
pickup_location = " Kilimani, Nairobi, Kenya"
destination_location = "8 Quarry Rd, Nairobi, Kenya"

geolocator = Nominatim(user_agent="patrick_app")
pikcup_coord = geolocator.geocode(str(pickup_location))
destination_coord = geolocator.geocode(str(destination_location))

pickup_lat = pikcup_coord.latitude
pickup_long = pikcup_coord.longitude


destination_lat = destination_coord.latitude
destination_long = destination_coord.longitude


In [11]:
pickup_lat = pikcup_coord.latitude
pickup_long = pikcup_coord.longitude

### Distance 

In [12]:
start = (pickup_lat, pickup_long)
end  = (destination_lat, destination_long)

In [13]:
end

(-1.2821967, 36.8361819)

In [14]:
distance_in_km = round(distance.distance(start, end).km  ,)

### Cluster

In [15]:
with open("/Users/patrickmacclenihan/Desktop/final_project/JP notebooks/pickup_cluster_model", 'rb') as file:
    pickup_kmean_model = pickle.load(file)

with open("/Users/patrickmacclenihan/Desktop/final_project/JP notebooks/destination_cluster_model", 'rb') as file:
    destination_kmean_model = pickle.load(file)

In [16]:
pickup_cluster = pickup_kmean_model.predict(np.array([pickup_lat,pickup_long]).reshape(1,-1)).astype(str)
destination_cluster = destination_kmean_model.predict(np.array([destination_lat,destination_long]).reshape(1,-1)).astype(str)



### PCA

In [17]:
with open("/Users/patrickmacclenihan/Desktop/final_project/JP notebooks/pca_model", 'rb') as file:
    pca = pickle.load(file)

In [18]:
long_lat = np.array([pickup_lat,pickup_long]).reshape(1,-1)

In [19]:
pickup_pca_0 = pca.transform(np.array([pickup_lat,pickup_long]).reshape(1,-1))[:, 0]
pickup_pca_1 = pca.transform(np.array([pickup_lat,pickup_long]).reshape(1,-1))[:, 1]
destination_pca_0 = pca.transform(np.array([destination_lat,destination_long]).reshape(1,-1))[:, 0]
destination_pca_1 = pca.transform(np.array([destination_lat,destination_long]).reshape(1,-1))[:, 1]

### With Uber 

In [20]:
polygons = pickle.load( open( "/Users/patrickmacclenihan/Desktop/final_project/DATA/DataFrame_for_maps", "rb" ) )

In [21]:
area_pickup = 1
for  row in polygons["geometry"] : 
    area_pickup +=1 
    if Point(pickup_lat, pickup_long).within(row) : 
        pickup_id  = area_pickup 

In [22]:
area_destination = 1 
for  row in polygons["geometry"] : 
    area_destination +=1 
    if Point(destination_lat, destination_long).within(row) : 
        destination_id = area_destination

In [23]:
pickup_id, destination_id

(58, 63)

In [24]:
##Speed
speed = pickle.load( open( "/Users/patrickmacclenihan/Desktop/final_project/JP notebooks/uber_speeds_aggregate.pickle", "rb" ) )


In [26]:
std_travel_time = speed[(speed["sourceid"] == pickup_id) & (speed["dstid"] == destination_id) & (speed["hod"] == pick_up_hour) & (speed["week_end_or_not"] == is_week_end(day_of_week))]["std_travel_time"].values
mean_travel_time = speed[(speed["sourceid"] == pickup_id) & (speed["dstid"] == destination_id) & (speed["hod"] == pick_up_hour) & (speed["week_end_or_not"] == is_week_end(day_of_week))]["mean_travel_time"].values


In [27]:
mean_travel_time

array([], dtype=float64)

In [28]:
std_travel_time

array([], dtype=float64)

In [55]:
new_dist_id = destination_id  
while len(std_travel_time.tolist()) == 0   : 
    new_dist_id += 1 
    std_travel_time = speed[(speed["sourceid"] == pickup_id) & (speed["dstid"] == new_dist_id ) & (speed["hod"] == pick_up_hour) & (speed["week_end_or_not"] == is_week_end(day_of_week))]["std_travel_time"].values

while len(mean_travel_time.tolist()) == 0   : 
    new_dist_id += 1 
    mean_travel_time = speed[(speed["sourceid"] == pickup_id) & (speed["dstid"] == new_dist_id) & (speed["hod"] == pick_up_hour) & (speed["week_end_or_not"] == is_week_end(day_of_week))]["mean_travel_time"].values

new_dist_id


63

In [56]:
new_dist_id

63

In [31]:
std_travel_time

array([1295.98])

## Riders info 

In [32]:
df_riders = pickle.load( open( "/Users/patrickmacclenihan/Desktop/final_project/DATA/riders_info_for_predict", "rb" ) )


In [33]:
rider = pd.DataFrame(df_riders.iloc[randint(0,df_riders.shape[0]),1::]).T

In [34]:
rider = rider.astype(int)

## Temperature 

In [35]:


url = "https://weatherbit-v1-mashape.p.rapidapi.com/current"

querystring = {"lang":"en","lon":"36.8219462","lat":"-1.2920659"}

headers = {
    'x-rapidapi-host': "weatherbit-v1-mashape.p.rapidapi.com",
    'x-rapidapi-key': "870480e1aamsh96e6e492e02203fp11faacjsndce42aecd5d0"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
request = json.loads(response.text)
temperature = request["data"][0]['temp']


# Creating DF

In [36]:
df_predict = pd.DataFrame(columns = ['Order No', 'User Id', 'Vehicle Type', 'Platform Type',
       'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time',
       'Arrival at Destination - Day of Month',
       'Arrival at Destination - Weekday (Mo = 1)',
       'Arrival at Destination - Time', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'Rider Id',
       'Time from Pickup to Arrival', 'No_Of_Orders', 'Age', 'Average_Rating',
       'No_of_Ratings', 'Pickup_Hour', 'Speed_in_km/h', 'Week_end',
       'pickup_long_lat', 'destination_long_lat', 'Pickup_Id',
       'Destination_Id', 'sourceid_x', 'dstid', 'hod_x', 'week_end_or_not_x',
       'std_travel_time', 'mean_travel_time', 'sourceid_y', 'hod_y',
       'week_end_or_not_y', 'min_travel_time', 'sourceid', 'hod',
       'week_end_or_not', 'min_std_travel_time', 'destination_clusters',
       'pickup_clusters', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'Rush_Hour', 'Average_Rider_Speed'])


In [37]:
for i in range(10) : 
    df_predict["Placement - Weekday (Mo = 1)"] = day_of_week
    df_predict["Placement - Day of Month"] = day
    df_predict['Distance (KM)'] = distance_in_km
    df_predict["Platform Type"] = platform_type
    df_predict["Personal or Business"] = Personnal_or_Business
    df_predict["Pickup Lat"] = pickup_lat
    df_predict["Pickup Long"] = pickup_long
    df_predict["Destination Lat"] = destination_lat
    df_predict["Destination Long"] = destination_long
    df_predict["sourceid"] = pickup_id
    df_predict["dstid"] = destination_id
    df_predict["Pickup_Hour"] = pick_up_hour
    df_predict["Week_end"] = week_end
    df_predict['std_travel_time'] = std_travel_time
    df_predict["mean_travel_time"] = mean_travel_time
    df_predict['pickup_pca0'] = pickup_pca_0
    df_predict['pickup_pca1'] = pickup_pca_1
    df_predict['dropoff_pca0'] = destination_pca_0
    df_predict['dropoff_pca1'] = destination_pca_1
    df_predict['pickup_clusters'] = pickup_cluster
    df_predict["destination_clusters"] = destination_cluster
    df_predict['Rush_Hour'] = rush_hour
    df_predict["Temperature"] = temperature 
    df_predict["Age"] = rider["Age"].values
    df_predict["Average_Rider_Speed"] = rider["Average_Rider_Speed"].values
    df_predict["No_Of_Orders"] = rider["No_Of_Orders"].values
    df_predict["Average_Rating"] = rider["Average_Rating"].values
    df_predict["No_of_Ratings"] = rider["No_of_Ratings"].values



## Dummies

In [38]:
to_dummies = ['Platform Type',
'Personal or Business',
'Placement - Day of Month',
'Pickup_Hour',
'Pickup_Id',
'Destination_Id',
'Week_end',
'destination_clusters',
'pickup_clusters',
'Rush_Hour',
'Placement - Weekday (Mo = 1)']


In [39]:

for col in to_dummies : 
    col_val = df_predict[col][0] 
    if col_val == 1 : 
        df_predict.rename(columns={col:f'{col}_2'}, inplace=True)
        df_predict[f'{col}_2'] = 0
    else : 
        df_predict.rename(columns={ col : f"{col}_{col_val}"},  inplace =True )
        df_predict[f"{col}_{col_val}"] = 1

In [40]:
columns = pickle.load( open( "/Users/patrickmacclenihan/Desktop/final_project/DATA/columns", "rb" ) )
columns_to_drop = pickle.load( open( "/Users/patrickmacclenihan/Desktop/final_project/DATA/columns_to_drop", "rb" ) )



In [41]:
for col in columns : 
    if col not in df_predict.columns: 
        df_predict[col] = 0 
        

In [42]:
len(df_predict.columns)

341

In [43]:
for col_to_drop in columns_to_drop : 
    if col_to_drop in df_predict.columns : 
        df_predict.drop(col_to_drop ,axis = 1 ,  inplace = True )

In [44]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 306 entries, Platform Type_0 to Destination_Id_72
dtypes: float64(7), int64(298), object(1)
memory usage: 2.5+ KB


In [45]:
categorical = [var for var in df_predict.columns if df_predict[var].dtype=='O']
print('There are {} categorical variables\n'.format(len(categorical)))
print("Categorical variables are  : \n", categorical)

There are 1 categorical variables

Categorical variables are  : 
 ['week_end_or_not']


In [46]:
for col in df_predict : 
    if col not in columns : 
        df_predict.drop(col, axis = 1) 

In [47]:
df_train = pickle.load( open( "/Users/patrickmacclenihan/Desktop/final_project/JP notebooks/train_data_ready", "rb" ) )


In [48]:
last_col_remove = []
for i in df_predict : 
    if i not in df_train : 
        last_col_remove.append(i)
last_col_remove

['Platform Type_0',
 'Personal or Business_2',
 'Placement - Weekday (Mo = 1)_0',
 'Week_end_0',
 'Pickup_Id_nan',
 'Destination_Id_nan',
 'Rush_Hour_0']

In [49]:
while len(df_predict.columns) >= 300 : 
    for col in last_col_remove : 
        df_predict.drop(col, axis = 1, inplace = True)
    

In [50]:
for i in df_predict : 
    if i not in df_train : 
        print(i)

In [51]:
df_predict.fillna(0, inplace = True )

In [52]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 299 entries, Placement - Day of Month_15 to Destination_Id_72
dtypes: float64(7), int64(292)
memory usage: 2.5 KB


# Predict

In [84]:
model = pickle.load( open("/Users/patrickmacclenihan/Desktop/final_project/JP notebooks/model_finalized", "rb" ) )

In [85]:
prediction = model.predict(df_predict)


In [139]:
prediction = round(model.predict(df_predict).tolist()[0]) / 60


In [60]:
pickup_up_time = datetime(int(year),int(month),int(day), int(pick_up_hour), int(pick_up_min)) 

In [67]:

from datetime import timedelta
timedelta(seconds=round(prediction.tolist()[0]) )

datetime.timedelta(seconds=2547)

In [79]:
delivery_time_date = datetime(int(year),int(month),int(day), int(pick_up_hour), int(pick_up_min))  +  timedelta(seconds=round(prediction.tolist()[0]) ) 
delivery_date = delivery_time_date.strftime("%m/%d/%Y")
delivery_time = delivery_time_date.strftime("%H:%M")

'Your parcel will be deliver on : 06/15/2020 at 15:12'

# Twilio Text

In [120]:
phone_number = '0688942346'
phone_number = '+33' + phone_number[1:]


In [121]:

from twilio.rest import Client


# Your Account Sid and Auth Token from twilio.com/console
# DANGER! This is insecure. See http://twil.io/secure
account_sid = '//'
auth_token = '//'
client = Client(account_sid, auth_token)


message = client.messages.create(
                              body='Hi there!',
                              from_='+12056971552',
                              to= phone_number
                          )

print(message.sid)


SMc9e16974f0ae4abe9becc6f1b8c2ea4c


# foluim Map

In [96]:
import folium 

In [100]:
pickup_location = " Kilimani, Nairobi, Kenya"
destination_location = "8 Quarry Rd, Nairobi, Kenya"

geolocator = Nominatim(user_agent="patrick_app")
pikcup_coord = geolocator.geocode(str(pickup_location))
destination_coord = geolocator.geocode(str(destination_location))


In [103]:
pikcup_coord[1][0]

-1.2874422

In [111]:
NAIROBI =(-1.2920659,36.8219462)
base_map= folium.Map(location= NAIROBI, min_zoom = 11,
                      zoom_start=11, tiles= 'cartodbpositron')


folium.CircleMarker(location=[pikcup_coord[1][0], pikcup_coord[1][1]],
                            radius=10,
                            tooltip=None,
                            color='#44b07aff',
                            fill_color='#44b07aff',
                            fill=True).add_to(base_map)       

folium.CircleMarker(location=[destination_coord[1][0], destination_coord[1][1]],
                            radius=10,
                            tooltip=None,
                            color='#11ABF9 ',
                            fill_color='#11ABF9 ',
                            fill=True).add_to(base_map)  

base_map 



In [114]:
base_map.save("Prediciton_Map.html")

In [115]:
import IPython
IPython.display.HTML(filename='Prediciton_Map.html')